# Import the Zenbase Library

In [ ]:
import sys
import subprocess

def install_package(package):
    try:
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])
    except subprocess.CalledProcessError as e:
        print(f"Failed to install {package}: {e}")
        raise

def install_packages(packages):
    for package in packages:
        install_package(package)

try:
    # Check if running in Google Colab
    import google.colab
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

if IN_COLAB:
    # Install the zenbase package if running in Google Colab
    # install_package('zenbase')
    # Install the zenbse package from a GitHub branch if running in Google Colab
    install_package('git+https://github.com/zenbase-ai/lib.git@main#egg=zenbase&subdirectory=py')

    # List of other packages to install in Google Colab
    additional_packages = [
        'python-dotenv',
        'langsmith[vcr]',
        'openai',
        'langchain',
        'langchain_openai'
    ]
    
    # Install additional packages
    install_packages(additional_packages)

# Now import the zenbase library
try:
    import zenbase
except ImportError as e:
    print("Failed to import zenbase: ", e)
    raise

# Configure the Environment

In [4]:
from pathlib import Path
from dotenv import load_dotenv

# import os
#
# os.environ["OPENAI_API_KEY"] = "..."
# os.environ["LANGCHAIN_API_KEY"] = "..."
# os.environ["LANGCHAIN_TRACING_V2"] = "true"

load_dotenv(Path("../../.env.test"), override=True)

True

In [5]:
import nest_asyncio

nest_asyncio.apply()

# Initial Setup

In [ ]:
from langsmith.wrappers import wrap_openai
from openai import OpenAI

openai = wrap_openai(OpenAI())

# What you already have should look like below:

## Your OpenAI Call should look like this with LangChain (It could be with OpenAI too, doesn't matter)

In [6]:
import json

from langsmith import traceable
from langsmith.schemas import Run, Example


# Define your LLM function
@traceable
def openai_json_response(inputs: dict) -> dict:
    messages = [
        {
            "role": "system",
            "content": "You are an expert math solver. Your answer must be just the number with no separators, and nothing else. Follow the format of the examples. Think step by step. Respond with a JSON object with a key of 'answer'.",
        },
        {"role": "user", "content": json.dumps(inputs)}
    ]

    response = openai.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages,
        response_format={"type": "json_object"},
    )

    return json.loads(response.choices[0].message.content)



## Your Scoring Function should look like this:

In [ ]:
# Define your Langsmith evaluator
def score_answer(run: Run, example: Example):
    match (answer := run.outputs["answer"]):
        case int():
            output = str(answer).strip()
        case str():
            output = answer.split("#### ")[-1].strip()
    target = example.outputs["answer"].split("#### ")[-1].strip()
    return {
        "key": "correctness",
        "score": int(output == target),
    }

## Your Evaluation should look like this:

In [7]:
# Evaluate using LangSmith
from langsmith import Client, evaluate

langsmith = Client()
evalset = list(langsmith.list_examples(dataset_name="GSM8K_test_set_langsmith_dataset_2ii5SKBzVHu3UVUmiFIxFxSvsFm"))

evaluate_kwargs = dict(
    data=evalset,
    evaluators=[score_answer],
    client=langsmith,
    max_concurrency=2,
)

evaluate(openai_json_response, **evaluate_kwargs)

View the evaluation results for experiment: 'spotless-plane-52' at:
https://smith.langchain.com/o/f145e0fe-631c-5153-984d-08acb624f83e/datasets/ab14f6ad-3e00-43b8-9ad6-3ce6bdb510f0/compare?selectedSessions=6bc92b05-dc54-4550-adb1-b5089251b497


0it [00:00, ?it/s]

<ExperimentResults spotless-plane-52>

# How you should do the few-shot learning

## Rewrite your langchain_chain function to use the `zenbase` decorators

In [16]:
from zenbase.types import LMRequest, deflm

# Wrap your existing chain with @deflm and take in a `LMRequest` object
# An LMRequest has the inputs for your chain and has a `zenbase` attribute.
# This `zenbase` attribute includes the fields that Zenbase optimises.

# LMRequest.inputs => LM function inputs
# LMRequest.zenbase => optimized LLM params

@deflm
@traceable
def openai_json_response(request: LMRequest) -> dict:
    messages = [
        {
            "role": "system",
            "content": "You are an expert math solver. Your answer must be just the number with no separators, and nothing else. Follow the format of the examples. Think step by step. Respond with a JSON object.",
        },
    ]

    for demo in request.zenbase.task_demos:
        messages += [
            {"role": "user", "content": json.dumps(demo.inputs)},
            {"role": "assistant", "content": json.dumps(demo.outputs)},
        ]
    messages.append({"role": "user", "content": json.dumps(request.inputs)})

    response = openai.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages,
        response_format={"type": "json_object"},
    )

    return json.loads(response.choices[0].message.content)

## Optimize the few-shot learning


### Define your optimizer:


In [18]:
from zenbase.adaptors.langchain import ZenLangSmith
from zenbase.optim.metric.labeled_few_shot import LabeledFewShot

demoset = ZenLangSmith.examples_to_demos(
    langsmith.list_examples(dataset_name="GSM8K_train_set_langsmith_dataset_2ii5SPh2qGRjlCzFIkp8d8qcKhH")
)
optimizer = LabeledFewShot(demoset=demoset, shots=3)


View the evaluation results for experiment: 'zenbase-right-sized-next-55ecd2a9' at:
https://smith.langchain.com/o/f145e0fe-631c-5153-984d-08acb624f83e/datasets/ab14f6ad-3e00-43b8-9ad6-3ce6bdb510f0/compare?selectedSessions=945be100-b92b-49fd-9e0d-81950c8c5ce6


0it [00:00, ?it/s]

View the evaluation results for experiment: 'zenbase-persevering-5thgeneration-7b5c3289' at:
https://smith.langchain.com/o/f145e0fe-631c-5153-984d-08acb624f83e/datasets/ab14f6ad-3e00-43b8-9ad6-3ce6bdb510f0/compare?selectedSessions=d42ddd91-5cb0-40ba-8128-3ec0591380aa


0it [00:00, ?it/s]

### Perform the optimization


In [ ]:
best_fn, candidates, _ = optimizer.perform(
    # Pass deflm decorated function
    openai_json_response,
    # Exactly the same as what you are passing to your evaluate function
    evaluator=ZenLangSmith.metric_evaluator(**evaluate_kwargs),
    samples=2,
    rounds=1,
)

### Use the best function

In [19]:
# Now you can use your zenbase fn
best_fn({"question": "If I have 30% of shares, and Mo has 24.5% of shares, how many of our 10M shares are unassigned?"})

{'answer': 'I have 30% shares which is 30/100 * 10M = <<30/100*10>>3000000 shares.\nMo has 24.5% shares which is 24.5/100 * 10M = <<24.5/100*10>>2450000 shares.\nTogether we have 3000000 + 2450000 = <<3000000+2450000=5450000>>5450000 shares.\nSo, unassigned shares are 10M - 5450000 = <<10000000-5450000=4550000>>4550000 shares.\n#### 4550000'}

### Inspect the best function

In [20]:
best_fn.zenbase.task_demos

(LMDemo(inputs={'question': 'Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?'}, outputs={'answer': 'Natalia sold 48/2 = <<48/2=24>>24 clips in May.\nNatalia sold 48+24 = <<48+24=72>>72 clips altogether in April and May.\n#### 72'}, original_object=Example(dataset_id=UUID('0a09e04d-e9b7-4684-8a8c-aa603af76146'), inputs={'question': 'Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?'}, outputs={'answer': 'Natalia sold 48/2 = <<48/2=24>>24 clips in May.\nNatalia sold 48+24 = <<48+24=72>>72 clips altogether in April and May.\n#### 72'}, metadata=None, id=UUID('685a1ba1-9e26-4495-a17a-7efcfcc04dc9'), created_at=datetime.datetime(2024, 7, 2, 22, 49, 15, 568348, tzinfo=datetime.timezone.utc), modified_at=datetime.datetime(2024, 7, 2, 22, 49, 15, 568348, tzinfo=datetime.timezone.utc)

### Save the best function

In [21]:
# You can also save the zenbase params for re-use
import pickle

pickled_zenbase = pickle.dumps(best_fn.zenbase)
openai_json_response.zenbase = pickle.loads(pickled_zenbase)

openai_json_response({"question": "What is 2 + 2?"}) # uses the best few-shot demos

{'answer': '4'}